<script async src="https://www.googletagmanager.com/gtag/js?id=UA-59152712-8"></script>
<script>
  window.dataLayer = window.dataLayer || [];
  function gtag(){dataLayer.push(arguments);}
  gtag('js', new Date());

  gtag('config', 'UA-59152712-8');
</script>

# `GRHayL`: GRMHD HLLE Flux Solver

## Author: Terrence Pierre Jacques

<a id='intro'></a>

**Notebook Status:** <font color=green><b> Validated </b></font>

**Validation Notes:** This code produces the expected C code for the generated functions.

## This module presents the functionality of [IGM_All_fluxes.py](/edit/fluxes/IGM_All_fluxes.py).

## Introduction: 
This notebook documents and self-validates the C code generation of the HLLE solver, [Harten, Lax, and von Leer](https://epubs.siam.org/doi/pdf/10.1137/1025002)  and [Einfeldt](https://epubs.siam.org/doi/10.1137/0725021), for the flux terms of our GRMHD formulation, generated NRPy+ to be used later by GRHayL. The C code documented and produced by this notebook assumes the all primitive variables and equation of state variables have been reconstructed to the left and right cell interfaces using some appropriate reconstruction algorithm, such as the piecewise-parabolic method (PPM) of [Colella and Woodward (1984)](https://crd.lbl.gov/assets/pubs_presos/AMCS/ANAG/A141984.pdf). The equations we will generate C code for are defined in [GRMHD_equations_new_version.py](/edit/GRMHD_formulation/GRMHD_equations_new_version.py), and documented in [Tutorial-GRMHD_Equations-Cartesian.ipynb](../GRMHD_formulation/Tutorial-GRMHD_Equations-Cartesian.ipynb). Note that since the GRHayL gems are designed to be infrastructure agnostic, the C codes generated here will not take derivatives of the fluxes, but will merely store their values within C structs.

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This notebook is organized as follows

1. [Step 1](#prelim): Preliminaries
1. [Step 2](#flux): GRMHD Flux Terms
    1. [Step 2.a](#flux_a): Calculate $T^{\mu\nu}$ and Related Quantities
    1. [Step 2.b](#flux_b): Calculate HLLE Fluxes
1. [Step 3](#C_code): C Code Generation
    1. [Step 3.a](#C_code_a): Define Input Variables and Access Structs
    1. [Step 3.b](#C_code_b): Printing the C Functions
1. [Step 4](#code_validation): Code Validation against `IGM_All_fluxes` Module
1. [Step 5](#latex_pdf_output): Output this notebook to $\LaTeX$-formatted PDF file

<a id='prelim'></a>

# Step 1: Preliminaries \[Back to [top](#toc)\]
$$\label{prelim}$$

This first block of code imports the core NRPy+ functionality after first adding the main NRPy+ directory to the path. 

In [1]:
# Step 0: Add NRPy's directory to the path
# https://stackoverflow.com/questions/16780014/import-file-from-parent-directory
import os,sys
GRMHD_dir_path = os.path.join("../GRMHD_formulation/")
sys.path.append(GRMHD_dir_path)
import GRMHD_equations_new_version as GRMHD    # NRPy+: Generate general relativistic magnetohydrodynamics equations

nrpy_dir_path = os.path.join("../nrpy/")
if nrpy_dir_path not in sys.path:
    sys.path.append(nrpy_dir_path)
    
from outputC import outputC, outCfunction, outC_function_dict # NRPy+: Core C code output module
import sympy as sp               # SymPy: The Python computer algebra package upon which NRPy+ depends
import NRPy_param_funcs as par   # NRPy+: Parameter interface
import indexedexp as ixp         # NRPy+: Symbolic indexed expression (e.g., tensors, vectors, etc.) support
import reference_metric as rfm   # NRPy+: Reference metric support
import cmdline_helper as cmd     # NRPy+: Multi-platform Python command-line interface

thismodule = "IGM-fluxes"

Ccodesdir = "IGM_standalone_Ccodes/"
cmd.mkdir(os.path.join(Ccodesdir))

#Step 0: Set the spatial dimension parameter to 3.
par.set_parval_from_str("grid::DIM", 3)
DIM = par.parval_from_str("grid::DIM")

CoordSystem = "Cartesian"

# Set coordinate system to dst_basis
par.set_parval_from_str("reference_metric::CoordSystem", CoordSystem)
rfm.reference_metric()

<a id='flux'></a>

# Step 2: GRMHD Flux Terms \[Back to [top](#toc)\]
$$\label{flux}$$


In this tutorial module we will compute the flux terms for the conservative variables $U=\left\{\rho_{\star},\tilde{\tau},\tilde{S}_{i}\right\}$ along all three flux directions, which are defined in terms of the primitive variables as

$$
\left(
\begin{matrix}
\rho_{\star}\\
\tilde{\tau}\\
\tilde{S}_{i}
\end{matrix}
\right)
=
\left(
\begin{matrix}
\alpha\sqrt{\gamma}\rho_{b}u^{0}\\
\alpha^{2}\sqrt{\gamma}T^{00}-\rho_{\star}\\
\left(\rho_{\star}h + \alpha u^{0}\sqrt{\gamma}b^{2}\right)u_{i} - \alpha\sqrt{\gamma}b^{0}b_{i}
\end{matrix}
\right)\ .
$$

The flux terms for these conservative variables are

$$
\boldsymbol{F} 
= 
\left(
\begin{matrix}
F^{i}_{\rho_{\star}}\\
F^{i}_{\tilde{\tau}}\\
\left(F_{\tilde{S}}\right)^{j}_{\ i}
\end{matrix}
\right)
=
\left(
\begin{matrix}
\rho_{\star}v^{i}\\
\alpha^{2}\sqrt{\gamma}T^{0j} - \rho_{\star}v^{j}\\
\alpha\sqrt{\gamma}T^{j}_{\ i}
\end{matrix}
\right)\ .
$$

The MHD flux algorithm computes, for each of the fluxes above, the standard Harten-Lax-van Leer (HLL) flux

$$
\boxed{F^{\rm HLL} = \frac{c^{-}F_{r} + c^{+}F_{l} - c^{+}c^{-}\left(U_{r} - U_{l}\right)}{c^{+} + c^{-}}}\ .
$$

Note that for the $\tilde{S}_i$ flux there is the added complication that we must keep track of each component, as well as each flux direction.


<a id='flux_a'></a>

## Step 2.a: Calculate $T^{\mu\nu}$ and Related Quantities \[Back to [top](#toc)\]
$$\label{flux_a}$$

Below we define a function to calculate the stress-energy tensor $T^{\mu\nu}$ and other related quantites to compute the compute the flux terms for each conservative variable. The flux terms will be functions of the reconstructed primitive variables and metric quantites interpolated to cell interfaces. We also define a function to calculate the HLLE fluxes:

$$
\boxed{F^{\rm HLL} = \frac{c^{-}F_{r} + c^{+}F_{l} - c^{+}c^{-}\left(U_{r} - U_{l}\right)}{c^{+} + c^{-}}}\ .
$$

Note that while the flux calculation relies on computations of the characteristic speeds $c^{+}$ and $c^{-}$, we calculate them in separate functions, using [IGM_Characteristic_Speeds.py](/edit/fluxes/IGM_Characteristic_Speeds.py), and documented in [Tutorial-IGM-Characteristic_Speeds.ipynb](../fluxes/Tutorial-IGM-Characteristic_Speeds.ipynb). This seperation is done to help sympy's cse algorithm.

In [2]:
# We'll rewrite this assuming that we've passed the entire reconstructed
# gridfunctions. You could also do this with only one point, but then you'd
# need to declare everything as a Cparam in NRPy+
    
def calculate_GRMHD_Tmunu_and_contractions(formalism, flux_dirn, mom_comp, 
                                          gammaDD,betaU,alpha,
                                          rho_b, P, h, u4U, BU):
    GRMHD.set_up_base_vars(formalism=formalism)
    
    GRMHD.compute_vU_from_u4U__no_speed_limit(u4U)
    
    # First compute stress-energy tensor T4UU and T4UD:
    GRMHD.compute_sqrtgammaDET(gammaDD)
    GRMHD.compute_smallb4U(gammaDD, betaU, alpha, u4U, BU, GRMHD.sqrt4pi)
    GRMHD.compute_smallbsquared(gammaDD, betaU, alpha, GRMHD.smallb4U)

    # First compute stress-energy tensor T4UU and T4UD:
    GRMHD.compute_T4UU(gammaDD,betaU,alpha, rho_b, P, h, u4U, GRMHD.smallb4U, GRMHD.smallbsquared)
    GRMHD.compute_T4UD(gammaDD,betaU,alpha, GRMHD.T4UU)
    
    # Compute conservative variables in terms of primitive variables
    GRMHD.compute_rho_star(alpha, GRMHD.sqrtgammaDET, rho_b,u4U)
    GRMHD.compute_tau_tilde(alpha, GRMHD.sqrtgammaDET, GRMHD.T4UU,GRMHD.rho_star)
    GRMHD.compute_S_tildeD(alpha, GRMHD.sqrtgammaDET, GRMHD.T4UD)

    # Next compute fluxes of conservative variables
    GRMHD.compute_rho_star_fluxU(GRMHD.VU,GRMHD.rho_star)
    GRMHD.compute_tau_tilde_fluxU(alpha, GRMHD.sqrtgammaDET, GRMHD.VU, GRMHD.T4UU, GRMHD.rho_star)
    GRMHD.compute_S_tilde_fluxUD (alpha, GRMHD.sqrtgammaDET, GRMHD.T4UD)

    global U_rho_star, F_rho_star
    U_rho_star = GRMHD.rho_star
    F_rho_star = GRMHD.rho_star_fluxU[flux_dirn]

    global U_tau_tilde, F_tau_tilde
    U_tau_tilde = GRMHD.tau_tilde
    F_tau_tilde = GRMHD.tau_tilde_fluxU[flux_dirn]

    global U_S_tilde, F_S_tilde
    U_S_tilde = GRMHD.S_tildeD[mom_comp]
    F_S_tilde = GRMHD.S_tilde_fluxUD[flux_dirn][mom_comp]

def HLLE_solver(cmax, cmin, Fr, Fl, Ur, Ul):
    # This solves the Riemann problem for the mom_comp component of the momentum
    # flux StildeD in the flux_dirn direction.

    # st_j_flux = (c_\min f_R + c_\max f_L - c_\min c_\max ( st_j_r - st_j_l )) / (c_\min + c_\max)
    return (cmin*Fr + cmax*Fl - cmin*cmax*(Ur-Ul) )/(cmax + cmin)


<a id='flux_b'></a>

## Step 2.b: Calculate HLLE Fluxes \[Back to [top](#toc)\]
$$\label{flux_b}$$

We now define functions to calculate the HLLE fluxes for each conserved variable, using information about the right and left interfaces.

In [3]:
def calculate_HLLE_fluxes(formalism, flux_dirn, alpha_face, gamma_faceDD, beta_faceU,
                          u4rU, u4lU, BrU, BlU,
                          rho_b_r, rho_b_l,
                          P_r, P_l,
                          h_r, h_l,
                          cmin, cmax):

    global Stilde_flux_HLLED, rho_star_HLLE_flux, tau_tilde_HLLE_flux
    Stilde_flux_HLLED = ixp.zerorank1()
#     rescaled_Stilde_fluxD = ixp.zerorank1()
    for mom_comp in range(3):
        calculate_GRMHD_Tmunu_and_contractions(formalism, flux_dirn, mom_comp, 
                                               gamma_faceDD,beta_faceU,
                                               alpha_face,
                                               rho_b_r, P_r, h_r, u4rU, 
                                               BrU)
        
        F_S_tilde_r = F_S_tilde
        U_S_tilde_r = U_S_tilde
        
        if mom_comp==0:
            U_rho_star_r = U_rho_star
            F_rho_star_r = F_rho_star

            U_tau_tilde_r = U_tau_tilde
            F_tau_tilde_r = F_tau_tilde            
        
        calculate_GRMHD_Tmunu_and_contractions(formalism, flux_dirn, mom_comp, 
                                               gamma_faceDD,beta_faceU,
                                               alpha_face,
                                               rho_b_l, P_l, h_l, u4lU, 
                                               BlU)
        
        F_S_tilde_l = F_S_tilde
        U_S_tilde_l = U_S_tilde
        
        if mom_comp==0:
            U_rho_star_l = U_rho_star
            F_rho_star_l = F_rho_star

            U_tau_tilde_l = U_tau_tilde
            F_tau_tilde_l = F_tau_tilde
            
            # now calculate HLLE derived fluxes, and rescale all of them
            rho_star_HLLE_flux = HLLE_solver(cmax, cmin, 
                                      F_rho_star_r, F_rho_star_l, 
                                      U_rho_star_r, U_rho_star_l)
            
            tau_tilde_HLLE_flux = HLLE_solver(cmax, cmin, 
                                      F_tau_tilde_r, F_tau_tilde_l, 
                                      U_tau_tilde_r, U_tau_tilde_l)
        
        # Rescale the flux term, to be FD
        Stilde_flux_HLLED[mom_comp] = HLLE_solver(cmax, cmin, 
                                      F_S_tilde_r, F_S_tilde_l, 
                                      U_S_tilde_r, U_S_tilde_l)

<a id='C_code'></a>

# Step 3: C Code Generation \[Back to [top](#toc)\]
$$\label{C_code}$$

In this section we now generate the C code to do these calculations.

<a id='C_code_a'></a>


# Step 3.a: Define Input Variables and Access Structs \[Back to [top](#toc)\]
$$\label{C_code_a}$$

Below we define our input variables, and generate C code for accessing the structs that store these variables.

In [4]:
# We will pass values of the gridfunction on the cell faces into the function. This requires us
# to declare them as C parameters in NRPy+. We will denote this with the _face infix/suffix.
formalism="ADM"

sqrt4pi = sp.symbols("SQRT_4_PI")
alpha_face = sp.symbols("alpha_face")
if formalism=="BSSN":
    cf_face = sp.symbols("cf_face")
    h_faceDD = ixp.declarerank2("h_faceDD","sym01",DIM=3)
    vet_faceU = ixp.declarerank1("vet_faceU", DIM=3)
    beta_faceU = vet_faceU
    
    import BSSN.ADM_in_terms_of_BSSN as AitoB
    AitoB.ADM_in_terms_of_BSSN()

    import BSSN.BSSN_quantities as Bq

    gamma_faceDD = ixp.zerorank2()
    for i in range(3):
        for j in range(3):
            gamma_faceDD[i][j] = AitoB.gammaDD[i][j].subs(Bq.hDD[i][j], h_faceDD[i][j]).subs(Bq.cf, cf_face)

else:
    beta_faceU = ixp.declarerank1("beta_faceU") 
    gamma_faceDD = ixp.declarerank2("gamma_faceDD","sym01")
               
# We'll need some more gridfunctions, now, to represent the reconstructions of BU and ValenciavU
# on the right and left faces
u4rU = ixp.declarerank1("u4rU", DIM=4)
u4lU = ixp.declarerank1("u4lU", DIM=4)

BrU = ixp.declarerank1("BrU")
BlU = ixp.declarerank1("BlU")

cmin_dirn0 = sp.symbols("cmin_dirn0")
cmin_dirn1 = sp.symbols("cmin_dirn1")
cmin_dirn2 = sp.symbols("cmin_dirn2")

cmax_dirn0 = sp.symbols("cmax_dirn0")
cmax_dirn1 = sp.symbols("cmax_dirn1")
cmax_dirn2 = sp.symbols("cmax_dirn2")

cmins = [cmin_dirn0, cmin_dirn1, cmin_dirn2]
cmaxs = [cmax_dirn0, cmax_dirn1, cmax_dirn2]

h_r = sp.symbols("h_r")
h_l = sp.symbols("h_l")

P_r = sp.symbols("P_r")
P_l = sp.symbols("P_l")

rho_b_r = sp.symbols("rhob_r")
rho_b_l = sp.symbols("rhob_l")

In [5]:
# GRHayL structs:

# typedef struct primitive_quantities {
# double rho, press, eps;
# double vx, vy, vz;
# double Bx, By, Bz;
# double entropy, Y_e, temperature;
# double enthalpy;
# } primitive_quantities;

# typedef struct conservative_quantities {
#   double rho, tau, Y_e;
#   double S_x, S_y, S_z;
#   double entropy;
# } conservative_quantities;

# typedef struct metric_quantities {
#   double adm_gxx, adm_gxy, adm_gxz;
#   double adm_gyy, adm_gyz, adm_gzz;
#   double adm_gupxx, adm_gupxy, adm_gupxz;
#   double adm_gupyy, adm_gupyz, adm_gupzz;
#   double betax, betay, betaz;
#   double lapse, lapseinv;
#   double psi2, psi4, psi6;
#   double psi4inv, lapseinv2;
#   double g4dn[4][4],g4up[4][4];
# } metric_quantities;

# typedef struct metric_derivatives {
#   double lapse[3];
#   double betax[3];
#   double betay[3];
#   double betaz[3];
#   double adm_gxx[3];
#   double adm_gxy[3];
#   double adm_gxz[3];
#   double adm_gyy[3];
#   double adm_gyz[3];
#   double adm_gzz[3];
# } metric_derivatives;

In [6]:
prims_NRPy_r = ["u4rU0", "u4rU1", "u4rU2", "u4rU3", "BrU0", "BrU1", "BrU2", "P_r", "rhob_r"]
prims_NRPy_l = ["u4lU0", "u4lU1", "u4lU2", "u4lU3", "BlU0", "BlU1", "BlU2", "P_l", "rhob_l"]

prims_GRHayL = ["ut", "vx", "vy", "vz", "Bx", "By", "Bz", "press", "rho"]

prestring = r"""
double h_r, h_l, cs2_r, cs2_l;

eos->compute_h_and_cs2(prims_r, &h_r, &cs2_r);
eos->compute_h_and_cs2(prims_l, &h_l, &cs2_l);
"""

for i in range(len(prims_NRPy_r)):
    if "v" in prims_GRHayL[i]: 
        vel_r_str = prims_NRPy_r[0]
        vel_l_str = prims_NRPy_l[0]
        prestring += "const double "+prims_NRPy_r[i]+" = prims_r->"+prims_GRHayL[i]+"*"+vel_r_str+";\n"
        prestring += "const double "+prims_NRPy_l[i]+" = prims_l->"+prims_GRHayL[i]+"*"+vel_l_str+";\n"
    else:
        prestring += "const double "+prims_NRPy_r[i]+" = prims_r->"+prims_GRHayL[i]+";\n"
        prestring += "const double "+prims_NRPy_l[i]+" = prims_l->"+prims_GRHayL[i]+";\n"

prestring += "const double "+str(alpha_face)+" = metric_face->lapse;\n"

checker = []

if formalism=="BSSN":
    prestring += "const double "+str(cf_face)+" = metric_face_quantities->"+str(cf_face)+";\n"

    for i in range(3):
        vetU_var = vet_faceU[i]
        prestring += "const double "+str(vetU_var)+" = metric_face_quantities->"+str(vetU_var)+";\n"

    for i in range(3):
        for j in range(3):
            hDD_var = h_faceDD[i][j]
            if hDD_var in checker:
                continue
            prestring += "const double "+str(hDD_var)+" = metric_face_quantities->"+str(hDD_var)+";\n"
            checker.append(hDD_var)

else:    
    prestring += "const double beta_faceU0 = metric_face->betax;\n"
    prestring += "const double beta_faceU1 = metric_face->betay;\n"
    prestring += "const double beta_faceU2 = metric_face->betaz;\n"

    prestring += "const double gamma_faceDD00 = metric_face->adm_gxx;\n"
    prestring += "const double gamma_faceDD01 = metric_face->adm_gxy;\n"
    prestring += "const double gamma_faceDD02 = metric_face->adm_gxz;\n"

    prestring += "const double gamma_faceDD11 = metric_face->adm_gyy;\n"
    prestring += "const double gamma_faceDD12 = metric_face->adm_gyz;\n"

    prestring += "const double gamma_faceDD22 = metric_face->adm_gzz;\n"

#     for i in range(3):
#             betaU_var = beta_faceU[i]
#             prestring += "const double "+str(betaU_var)+" = metric_face_quantities->"+str(betaU_var)+";\n"

#     for i in range(3):
#         for j in range(3):
#             gammaDD_var = gamma_faceDD[i][j]
#             if gammaDD_var in checker:
#                 continue
#             prestring += "const double "+str(gammaDD_var)+" = metric_face_quantities->"+str(gammaDD_var)+";\n"
#             checker.append(gammaDD_var)

<a id='C_code_b'></a>

## Step 3.b: Printing the C Functions \[Back to [top](#toc)\]
$$\label{C_code_b}$$

Here we now print out the C function that will be used in a point-wise fashion to calculate the fluxes. Note that the flux calculations differ for each direction, so we generate 3 C functions.

In [7]:
outCparams = "outCverbose=False,CSE_sorting=True"
# write_cmax_cmin=True

vars_to_write = ["cons->S_x", "cons->S_y", "cons->S_z", "cons->rho", "cons->tau"]
c_type = "void"

params  =  "const primitive_quantities *restrict prims_r, "
params  += "const primitive_quantities *restrict prims_l, "
params  += "const eos_parameters *restrict eos, "
params  += "const metric_quantities *restrict metric_face, "
params  += "conservative_quantities *restrict cons"

for flux_dirn in range(3):
    cmin_cmax_str = "double "+str(cmins[flux_dirn])+", "+str(cmaxs[flux_dirn])+";\n"
    
    calc_char_speeds_params_str = "(prims_r, prims_l, eos, metric_face, &"+str(cmins[flux_dirn])+", &"+str(cmaxs[flux_dirn])+")"

    calc_char_speeds_func_str = "calculate_characteristic_speed_dirn" + str(flux_dirn)

    calculate_HLLE_fluxes(formalism, flux_dirn, alpha_face, gamma_faceDD, beta_faceU,
                          u4rU, u4lU, BrU, BlU,
                          rho_b_r, rho_b_l,
                          P_r, P_l,
                          h_r, h_l,
                          cmins[flux_dirn], cmaxs[flux_dirn])

    vars_rhs = [Stilde_flux_HLLED[0], 
                Stilde_flux_HLLED[1], 
                Stilde_flux_HLLED[2], 
                rho_star_HLLE_flux,
                tau_tilde_HLLE_flux]

    body = outputC(vars_rhs, vars_to_write, params=outCparams, 
               filename="returnstring", prestring=(cmin_cmax_str+
                                                   calc_char_speeds_func_str+
                                                   calc_char_speeds_params_str+";\n\n"+
                                                   prestring))

    desc = "Compute the HLLE-derived fluxes on the left face in the " + str(flux_dirn) + "direction for all components."
    name = "calculate_HLLE_fluxes_dirn" + str(flux_dirn)
    includes = ["NRPy_basic_defines.h"]
    
    outCfunction(
        outfile=os.path.join(Ccodesdir,name+".c"),
        includes=includes,
        desc=desc,
        name=name,
        params=params,
        body= body, 
        enableCparameters=False)

Output C function calculate_HLLE_fluxes_dirn0() to file IGM_standalone_Ccodes/calculate_HLLE_fluxes_dirn0.c
Output C function calculate_HLLE_fluxes_dirn1() to file IGM_standalone_Ccodes/calculate_HLLE_fluxes_dirn1.c
Output C function calculate_HLLE_fluxes_dirn2() to file IGM_standalone_Ccodes/calculate_HLLE_fluxes_dirn2.c


<a id='code_validation'></a>

# Step 4:  Code Validation against `IGM_All_fluxes`  Module \[Back to [top](#toc)\]
$$\label{code_validation}$$


Here, as a code validation check, we verify agreement in the C code generated by 
1. this tutorial and 
2. the  [IGM_All_fluxes.py](/edit/fluxes/IGM_All_fluxes.py) module.

In [8]:
files = ["calculate_HLLE_fluxes_dirn0.c",
         "calculate_HLLE_fluxes_dirn1.c",
         "calculate_HLLE_fluxes_dirn2.c"]

In [9]:
# Define the directory that we wish to validate against:
valdir = "IGM_Ccode_library/"
cmd.mkdir(os.path.join(valdir))

import IGM_All_fluxes as fluxes
fluxes.Cfunction__GRMHD_fluxes(valdir, includes=includes,
                                       formalism=formalism,
                                       outCparams=outCparams)

import difflib
import sys

print("Printing difference between original C code and this code...")
# Open the files to compare

for file in files:
    print("Checking file " + file)
    with open(os.path.join(valdir,file)) as file1, open(os.path.join(Ccodesdir,file)) as file2:
        # Read the lines of each file
        file1_lines = file1.readlines()
        file2_lines = file2.readlines()
        num_diffs = 0
        for line in difflib.unified_diff(file1_lines, file2_lines, fromfile=os.path.join(valdir+file), tofile=os.path.join(Ccodesdir+file)):
            sys.stdout.writelines(line)
            num_diffs = num_diffs + 1
        if num_diffs == 0:
            print("No difference. TEST PASSED!")
        else:
            print("ERROR: Disagreement found with .c file. See differences above.")
            sys.exit(1)

Output C function calculate_HLLE_fluxes_dirn0() to file IGM_Ccode_library/calculate_HLLE_fluxes_dirn0.c
Output C function calculate_HLLE_fluxes_dirn1() to file IGM_Ccode_library/calculate_HLLE_fluxes_dirn1.c
Output C function calculate_HLLE_fluxes_dirn2() to file IGM_Ccode_library/calculate_HLLE_fluxes_dirn2.c
Printing difference between original C code and this code...
Checking file calculate_HLLE_fluxes_dirn0.c
No difference. TEST PASSED!
Checking file calculate_HLLE_fluxes_dirn1.c
No difference. TEST PASSED!
Checking file calculate_HLLE_fluxes_dirn2.c
No difference. TEST PASSED!


<a id='latex_pdf_output'></a>

# Step 5: Output this notebook to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

The following code cell converts this Jupyter notebook into a proper, clickable $\LaTeX$-formatted PDF file. After the cell is successfully run, the generated PDF may be found in the root NRPy+ tutorial directory, with filename
[Tutorial-GiRaFFE_NRPy-Stilde-flux.pdf](Tutorial-GiRaFFE_NRPy-Stilde-flux.pdf) (Note that clicking on this link may not work; you may need to open the PDF file through another means.)

In [10]:
import cmdline_helper as cmd    # NRPy+: Multi-platform Python command-line interface
cmd.output_Jupyter_notebook_to_LaTeXed_PDF("Tutorial-GRHayL-All_Fluxes_NRPy_Gen")

Traceback (most recent call last):
  File "/home/terrence/environments/ve3.10/lib/python3.10/site-packages/traitlets/traitlets.py", line 537, in get
    value = obj._trait_values[self.name]
KeyError: 'template_paths'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/home/terrence/environments/ve3.10/bin/jupyter-nbconvert", line 8, in <module>
    sys.exit(main())
  File "/home/terrence/environments/ve3.10/lib/python3.10/site-packages/jupyter_core/application.py", line 264, in launch_instance
    return super(JupyterApp, cls).launch_instance(argv=argv, **kwargs)
  File "/home/terrence/environments/ve3.10/lib/python3.10/site-packages/traitlets/config/application.py", line 846, in launch_instance
    app.start()
  File "/home/terrence/environments/ve3.10/lib/python3.10/site-packages/nbconvert/nbconvertapp.py", line 369, in start
    self.convert_notebooks()
  File "/home/terrence/environments/ve3.10/lib/python3.10/site-packag